# Derivatives and finite differences

<!-- <a href="http://depts.washington.edu/astron/research/n-body-shop/" target="_blank"><img src="img/cosmo25.HR.gold.jpg" width=400px/></a> -->
<a href="http://depts.washington.edu/astron/research/n-body-shop/" target="_blank"><img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/cosmo25.HR.gold.jpg" width=400px/></a>

## PHYS 2600

## Lecture 13

### Discrete derivatives

From your calculus textbook, a derivative is defined as follows:

$$
\frac{df}{dx} = \lim_{\epsilon \rightarrow 0} \frac{f(x+\epsilon) - f(x)}{\epsilon}.
$$

In fact, the derivative is often first introduced in terms of a __secant line__, which describes the slope of $f(x)$ between any two points $x_1$ and $x_2$:

$$
b = \frac{f(x_2) - f(x_1)}{x_2 - x_1},
$$

and we get the derivative as the slope of the __tangent line__ as $x_2$ is brought together with $x_1$.  

<!-- <img src="img/sec-to-tan.gif" width=400px /> -->
<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/sec-to-tan.gif" width=400px />

(Nice animation borrowed from [this UCSB page on calculus](http://clas.sa.ucsb.edu/staff/lee/Secant,%20Tangent,%20and%20Derivatives.htm).)



This gives an obvious way to define a discrete approximation of $df/dx$: just use the secant definition!

$$
f'(x_i) \approx y'_i = \frac{y_{i+1} - y_i}{x_{i+1} - x_i} = \frac{y_{i+1} - y_i}{a}
$$

where the second `=` is true when our points have constant spacing $a$.  $y'_i$ is a _discrete approximation_ to $f'(x_i)$.

Note that discretization we use is a *choice*.  We could have instead done this:

$$
f'(x_i) \approx y'_i = \frac{y_i - y_{i-1}}{a}.
$$

This is called the __backward difference__ definition; the one we started with was the __forward difference__.  They are identical (and equal to $f'(x_i)$) when $a \rightarrow 0$.

In fact, these aren't the only options - there are many ways to build a secant line!  We can also use a __central difference__:

$$
f'(x_i) \approx y'_i = \frac{y_{i+1} - y_{i-1}}{2a}
$$

(note the 2 in the denominator.)  This also becomes the usual derivative as the spacing $a$ between the points vanishes.

In fact, this version is often better: the _discretization error_ $f'(x_i) - y'_i$ is proportional to $a^2$, versus $a$ for forward/backward difference.  (The jargon you will see is that this version is __second-order accurate__.) 

There are approximations with even smaller discretization error, but they require using more than two points at once.  I won't go into them here - again, the best way to reduce error is just to make $a$ smaller, unless your problem is very expensive!

The `np.gradient()` function calculates the second-order accurate version of $y'_i$:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


x = np.linspace(0,2*np.pi,200)
dx = x[1]-x[0]
y = np.sin(x)
dy_dx = np.gradient(y, x[1]-x[0]) # dy/dx = cos(x)
plt.plot(x, y, color='k')
plt.plot(x, dy_dx, color='r')

Before we move on, a general comment: _you are better off calculating $f'(x)$ analytically_ whenever you can!  Numerical derivatives e.g. `np.gradient()` are for situations where you _can't_ do this - $f(x)$ is a "black box", or is just very complicated.  (Solving differential equations is an example where you don't know $f(x)$ in advance!)

There is a growing interest in __automatic differentiation__, which calculates _analytic_ derivatives from your Python functions: I won't talk more about this, but have a look at [JAX](https://jax.readthedocs.io/en/latest/) if you're interested.  (JAX is a foundational library for machine learning.)

## Discrete differential equations

Consider a first-order ODE of the form:

$$
\frac{du}{dt} = F(u(t), t)
$$

With the initial condition $u(0) = c$. This type of differential equation is known as an **initial value problem** (IVP).

If the right-hand side is just $F(t)$, then the solution is simple:

$$
u(t) = c +  \int_0^t d\tau\ F(\tau)
$$

i.e. the problem just reduces to a numerical integral.  The case where $u(t)$ appears on the right-hand side is _much_ more interesting.

To solve more generally, we discretize!  Replacing $t \rightarrow \{t_i\}$, the ODE becomes
$$
u'(t_i) = F(u_i, t_i)
$$
where we define $u_i = u(t_i)$.  


We don't know $u'(t)$, but we can approximate it with one of our discrete derivatives! Replacing $u'(t)$ with the forward difference gives us:
$$
\frac{u_{i+1} - u_i}{\Delta t} = F(u_i, t_i)
$$
This is now a __difference equation__ (discrete verison of a differential equation.) Note that we had to **choose** which finite difference approximation to use. Different approximations of $u'(t)$ give different difference equations. Replacing the derivative with a forward difference is called the "__forward Euler method__" or "__explicit Euler method__".


At first glance, the difference equation might not seem any better than the original differential equation, but rearranging terms gives us a formula
$$
u_{i+1} = u_i + F(u_i, t_i) \Delta t 
$$
for $u_{i+1}$ (the function evaluated at $t_{i+1}=t_{i} + \Delta t$) in terms of $u_i$ (the function evaluated at $t_{i}$). In other words, it gives us a way to compute future values of the function future from past values.

How can we use this to (approximately) solve the IVP? Remember our initial condition: 
$$
u_0=c
$$
Plugging this in lets us determine $u_1$, then plugging in $u_1$ gives $u_2$...
$$
u_1 = u_0 + F(u_0, t_0) \Delta t
...
$$
$$
u_2 = u_1 + F(u_1, t_1) \Delta t
$$

and so on!  Each step in this process is called a **timestep**

To reiterate, here is the full forward Euler method algorithm for solving $u'(t) = F(u(t), t)$:

__(1)__ Discretize time $t \rightarrow \{t_i\}$ and replace $u'(t)$ with the forward-difference $(u_{i+1} - u_i) / \Delta t$.  This gives:

$$
u_{i+1} = u_i + F(u_i, t_i)\ \Delta t
$$

__(2)__ Iterate over the range of all $\{t_i\}$, starting at $t_0$.  Accumulate an array or list of results $\{u_i\}$:
  - For each $t_i$, use the discrete equation with $u_i$ to find $u_{i+1}$.

__(3)__ Return the result for $\{u_i\}$, which is a discrete approximation to the solution $u(t)$ over the discrete interval $\{t_i\}$.

The sort of timestepping process is common to most numerical IVP solvers! Variations mostly just involve changing the finite difference formulas used to approximate derivatives.

In general, the quality of our numerical solution will depend on both the **stepsize** $\Delta t$ and on the specific finite difference formulas used to approximate derivatives. There are plenty of numerical schemes which outperform forward Euler, but Euler often does fine if you use a small enough timestep. It is always a good idea to **start with the simple approach** and only resort to more sophisticated methods when the simple method fails.

If Forward Euler doesn't seem to be working for a problem, you may want to try one of the higher order [__Runge-Kutta__ methods](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods). They're a bit trickier to implement, but they also tend to give much better approximations. We're not going to dig into the gory details of implementing more sophisticated ODE solvers our selves (yet!). The [scipy module](https://docs.scipy.org/doc/scipy/reference/integrate.html) already has a lot of the RK methods pre-built and ready to use!



First-order might seem kind of restrictive: what about dealing with higher-order equations, like we might see in kinematics?
$$
F(t) = m \frac{d^2x}{dt^2} \Rightarrow \frac{d^2x}{dt^2} =  \frac{F(t)}{m}
$$
Using a finite difference approximation of the second derivative is one option, but there are many others!

Another approach is exactly what we do on paper: we can work directly with the velocity $v = dx/dt$! This turns our **second order ODE in one variable** into a **first order ODE in two variables**:

$$
\frac{dv}{dt} = \frac{F(t)}{m}
$$
$$
\frac{dx}{dt} = v(t)
$$

This trick works for __any__ system of ODEs! You can always get rid of higher order derivatives in an ODE by introducing new variables.


Replacing $\frac{dv}{dt}$ and $\frac{dx}{dt}$ with forward difference approximations gives a pair of difference equation:
$$
 v_{i+1} = v_i + \frac{F_i}{m} \Delta t
$$
$$
 x_{i+1} = x_i + v_i \Delta t
$$
This sort of system can be solved using the same timestepping method from the previous example: start at $i=0$, with two initial conditions $(x_0, v_0)$, solve both equations to find $x_1$ and $v_1$, then repeat.

## Tutorial 13

Open up `tut13`, and we'll work through a numerical solution step by step.